<a href="https://colab.research.google.com/github/rajeshkpandey/rajeshkpandey.github.io/blob/main/Job_Title_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
print(os.listdir("./"))

import pandas as pd
import numpy as np
from scipy.stats import randint
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics

['.ipynb_checkpoints', '.ipython', '.jupyter', '.matplotlib', '3D Objects', 'anaconda3', 'AppData', 'Application Data', 'Contacts', 'Cookies', 'Documents', 'Downloads', 'Dropbox', 'Favorites', 'Job_Title_Prediction.ipynb', 'Job_Title_Prediction_9000.ipynb', 'Links', 'Local Settings', 'Music', 'My Documents', 'NetHood', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{53b39e87-18c4-11ea-a811-000d3aa4692b}.TxR.0.regtrans-ms', 'NTUSER.DAT{53b39e87-18c4-11ea-a811-000d3aa4692b}.TxR.1.regtrans-ms', 'NTUSER.DAT{53b39e87-18c4-11ea-a811-000d3aa4692b}.TxR.2.regtrans-ms', 'NTUSER.DAT{53b39e87-18c4-11ea-a811-000d3aa4692b}.TxR.blf', 'NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TM.blf', 'NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TMContainer00000000000000000001.regtrans-ms', 'NTUSER.DAT{53b39e88-18c4-11ea-a811-000d3aa4692b}.TMContainer00000000000000000002.regtrans-ms', 'ntuser.ini', 'OneDrive', 'PrintHood', 'Recent', 'Saved Games', 'Searches', 'SendTo', 'Start Menu', 'Templ

In [ ]:
df = pd.read_csv('train.csv')
df.dropna()
df.head()

,Job ID,Job Description,Title
0,JOB0002000,Description Are you interested in the opport...,HAI Prevention Data Analyst
1,JOB0002001,Instructional Designer needed for a contract o...,Instructional Designer
2,JOB0002002,Faith Church Office ManagerPosition Focus Shee...,Office Manager
3,JOB0002003,NO C2C FOR THIS REQUIREMENT Our LARGE and stab...,ETL Developer
4,JOB0002004,Event Promotions: Marketing / Advertising / Pu...,Event Promotions: Marketing / Advertising / Pu...


In [ ]:
test_df = pd.read_csv('test.csv')
test_df.dropna()
test_df.head()

,Job ID,Job Description,Title
0,JOB0000,TeamSoft is seeing an IT Support Specialist to...,NaN
1,JOB0001,The Wisconsin State Journal is seeking a flexi...,NaN
2,JOB0002,Report this job About the Job DePuy Synthes Co...,NaN
3,JOB0003,Why Join Altec? If you’re considering a career...,NaN
4,JOB0004,Position ID# 76162 # Positions 1 State CT C...,NaN


In [ ]:
df.shape


(20000, 3)

In [ ]:
test_df.shape

(2000, 3)

In [ ]:
df.tail()

,Job ID,Job Description,Title
19995,JOB00021995,This is a major premier Cincinnati based finan...,Assistant Vice President
19996,JOB00021996,Luxury homebuilder in Cincinnati seeking multi...,Accountant
19997,JOB00021997,RE: Adobe AEM- Client - Loca...,AEM/CQ developer
19998,JOB00021998,Jernberg Industries was established in 1937 an...,Electrician
19999,JOB00021999,Contract AdministratorCan you be the point per...,Contract Administrator


In [ ]:
test_df.tail()

,Job ID,Job Description,Title
1995,JOB0001995,GENERAL DESCRIPTIONResponsible for ensuring th...,NaN
1996,JOB0001996,Job ID: 94448Non-Exempt Oldcastle Precast i...,NaN
1997,JOB0001997,GENERAL DESCRIPTIONIs responsible for all phas...,NaN
1998,JOB0001998,Report this job About the Job Skilled Nursing ...,NaN
1999,JOB0001999,Job ID: 99428 Georgia Masonry Supply is a di...,NaN


In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\urmi8\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\urmi8\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#cleaning training dataset
for i in range(len(df)):
  df['Job Description'][i] = df['Job Description'][i].lower()
  df['Job Description'][i]="".join(char for char in df['Job Description'][i] if char not in string.punctuation)
  ps = PorterStemmer()
  wml = WordNetLemmatizer()
  df['Job Description'][i]=ps.stem(df['Job Description'][i])
  df['Job Description'][i]=wml.lemmatize(df['Job Description'][i])
  df['Job Description'][i]=df['Job Description'][i].split()
  filter_words = []
  Stopwords = set(stopwords.words('english'))
  for word in df['Job Description'][i]:
    if word not in Stopwords:
         filter_words.append(word)
  df['Job Description'][i] = filter_words 
  df['Job Description'][i] = " ".join(df['Job Description'][i])

In [ ]:
#cleaning testing set
for i in range(len(test_df)):
  test_df['Job Description'][i] = test_df['Job Description'][i].lower()
  test_df['Job Description'][i]="".join(char for char in test_df['Job Description'][i] if char not in string.punctuation)
  ps = PorterStemmer()
  wml = WordNetLemmatizer()
  test_df['Job Description'][i]=ps.stem(test_df['Job Description'][i])
  test_df['Job Description'][i]=wml.lemmatize(test_df['Job Description'][i])
  test_df['Job Description'][i]=test_df['Job Description'][i].split()
  filter_words = []
  Stopwords = set(stopwords.words('english'))
  for word in test_df['Job Description'][i]:
    if word not in Stopwords:
         filter_words.append(word)
  test_df['Job Description'][i] = filter_words
  test_df['Job Description'][i] = " ".join(test_df['Job Description'][i])

C:\Users\urmi8\AppData\Local\Temp/ipykernel_6460/3920492779.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Job Description'][i] = test_df['Job Description'][i].lower()
C:\Users\urmi8\AppData\Local\Temp/ipykernel_6460/3920492779.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Job Description'][i]="".join(char for char in test_df['Job Description'][i] if char not in string.punctuation)
C:\Users\urmi8\AppData\Local\Temp/ipykernel_6460/3920492779.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

In [ ]:
df['Title'][0].lower()

'hai prevention data analyst'

In [ ]:
#cleaning Response Variable i.e. Title for training data (lowercasing and removing punctuation)
for i in range(len(df)):
    df['Title'][i]=str(df['Title'][i]).lower()
    df['Title'][i]="".join(char for char in df['Title'][i] if char not in string.punctuation)

In [ ]:
#observing the data and its class count
temp_df=df.groupby('Title').count()

In [ ]:
temp_df.columns

Index(['Job ID', 'Job Description'], dtype='object')

In [ ]:
temp_df.shape[0]

10539

In [ ]:
temp_df['Job Description'].min()

1

In [ ]:
temp_df['Job Description'].max()

329

In [ ]:
count=0
for i in range(1,330):
    print(len(temp_df[temp_df['Job Description']==i]))
    count+=len(temp_df[temp_df['Job Description']==i])
print(count)

8396
1150
339
172
100
65
50
24
24
35
21
22
10
12
9
5
6
5
8
3
7
4
1
5
1
8
4
1
4
1
4
3
2
1
2
0
1
1
0
0
1
3
0
2
1
0
2
0
0
1
1
1
0
0
1
1
1
0
1
1
1
1
0
0
0
0
0
0
1
0
0
0
1
0
0
0
1
0
0
0
1
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
1
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
10539


In [ ]:
# Create a new dataframe with two columns
df1 = df[['Job Description', 'Title']].copy()

# Remove missing values (NaN)
df1 = df1[pd.notnull(df1['Job Description'])]

# Renaming job description column for a simpler name
df1.columns = ['Job_Description', 'Title'] 

df1.shape

(20000, 2)

In [ ]:
# Percentage of Job Description with text
total = df1['Job_Description'].notnull().sum()
round((total/len(df)*100),1)

100.0

In [ ]:
pd.DataFrame(df.Title.unique()).values

array([['hai prevention data analyst'],
       ['instructional designer'],
       ['office manager'],
       ...,
       ['patient access representative'],
       ['assistant vice president'],
       ['aemcq developer']], dtype=object)

In [ ]:
len(df.Title.unique())

10539

In [ ]:
df2 = df1.sample(9000, random_state=1).copy()


In [ ]:
pd.DataFrame(df2.Title.unique())

,0
0,construction management
1,certified medical assistant
2,associate attorney
3,monster
4,physical therapist
...,...
5498,cybercoders job application for hotel construc...
5499,slitter assistant
5500,rd shift production supervisor
5501,bakery clerk parttime canfield giant eagle


In [ ]:
# Create a new column 'category_id' with encoded categories 
df2['category_id'] = df2['Title'].factorize()[0]
category_id_df = df2[['Title', 'category_id']].drop_duplicates()


# Dictionaries for future use
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Title']].values)

# New dataframe head
df2.head()

,Job_Description,Title,category_id
11456,construction management – project managerslam ...,construction management,0
16528,certified medical assistant healthcare medical...,certified medical assistant,1
3253,established scottsdale general practice law fi...,associate attorney,2
18614,job title data architectlocation austintx• ext...,monster,3
1544,job description physical therapist responsibil...,physical therapist,4


In [ ]:
# New dataframe tail
df2.tail()

,Job_Description,Title,category_id
8190,join tosca team key role within salinas plant ...,rd shift production supervisor,5500
19715,sears roebuck co member sears holdings corpora...,consultative sales associate – home appliances,5000
12845,position summary together improve peoples ever...,bakery clerk parttime canfield giant eagle,5501
13849,business analyst project managerbusiness analy...,business analyst project manager,5502
12185,electrical engineer 110k 120k great benefits s...,electrical engineer,784


In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1,2), 
                        stop_words='english')

# We transform each Job Description into a vector
features = tfidf.fit_transform(df2.Job_Description).toarray()

labels = df2.category_id

print("Each of the %d job descriptions is represented by %d features (TF-IDF score of unigrams and bigrams)" %(features.shape))

Each of the 9000 job descriptions is represented by 73519 features (TF-IDF score of unigrams and bigrams)


In [ ]:
models = [
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
    LinearSVC(),
    #MultinomialNB(),
    #LogisticRegression(random_state=0),
]

# 3 Cross-validation
CV = 3
cv_df = pd.DataFrame(index=range(CV * len(models)))

entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

C:\Users\urmi8\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\urmi8\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


In [ ]:
mean_accuracy = cv_df.groupby('model_name').accuracy.mean()
std_accuracy = cv_df.groupby('model_name').accuracy.std()

acc = pd.concat([mean_accuracy, std_accuracy], axis= 1, 
          ignore_index=True)
acc.columns = ['Mean Accuracy', 'Standard deviation']
acc

,Mean Accuracy,Standard deviation
model_name,,
LinearSVC,0.292444,0.005719
RandomForestClassifier,0.083778,0.002037


In [ ]:
X = df2['Job_Description'] # Collection of Job descriptions
y = df2['Title'] # Target or the labels we want to predict

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    random_state = 0)

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words='english')

fitted_vectorizer = tfidf.fit(X_train)
tfidf_vectorizer_vectors = fitted_vectorizer.transform(X_train)

model = LinearSVC().fit(tfidf_vectorizer_vectors, y_train)

In [ ]:
predictions=[]
for i in range(len(test_df)):
  predictions.append(model.predict(fitted_vectorizer.transform([test_df['Job Description'][i]])))

In [ ]:
print(predictions)

[array(['digital marketing project manager'], dtype=object), array(['district service manager assistant'], dtype=object), array(['monster'], dtype=object), array(['supplier quality engineer'], dtype=object), array(['shift supervisor'], dtype=object), array(['tmobjp'], dtype=object), array(['monster'], dtype=object), array(['accounts payable clerk'], dtype=object), array(['server'], dtype=object), array(['body shop manager'], dtype=object), array(['aflac insurance sales agent'], dtype=object), array(['sales associate'], dtype=object), array(['graphic designer'], dtype=object), array(['teddy bear christian child development center job application for teachers needed  monstercom var monslogvars  jobid'],
      dtype=object), array(['industrial construction superintendent'], dtype=object), array(['engineer i  engineer ii  construction project administrator'],
      dtype=object), array(['n horizontal construction engineers'], dtype=object), array(['pest control technician – field service r

In [ ]:
prediction_df = pd.DataFrame(predictions, columns=['Title'])


In [ ]:
prediction_df.to_csv('predictions.csv',sep=',',index=True)